In [1]:
!nvidia-smi

Tue Jul  4 19:45:55 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti      On | 00000000:26:00.0 Off |                  N/A |
|  0%   54C    P8               15W / 200W|     15MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [99]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q tensorboard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [1]:
from datasets import load_dataset

data = load_dataset("squarelike/ko_medical_chat")

/home/issac/anaconda3/envs/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset json (/home/issac/.cache/huggingface/datasets/json/default-95ad2bd7d3162001/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 836.02it/s]


In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations'],
        num_rows: 3038
    })
})

In [3]:
data = data.map(
    lambda x: {
        'text': "\n".join([f"{'환자' if line['from']=='client' else '의사'}: {line['value']}{'</끝>' if line['from']!='client' else ''}" for line in x['conversations']])
      }
)

Loading cached processed dataset at /home/issac/.cache/huggingface/datasets/json/default-95ad2bd7d3162001/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-ba47b88b4c6a0c5e.arrow


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_id = "squarelike/llama2-ko-medical-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.68s/it]


In [5]:
data = data.map(lambda samples: tokenizer(samples["text"], truncation=True, max_length=2048), batched=True)

Loading cached processed dataset at /home/issac/.cache/huggingface/datasets/json/default-95ad2bd7d3162001/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-714327b44749993e.arrow


In [6]:
data['train'][1000]['text']

'환자: 밤에 자러 갈 때 허벅지 중앙을 따라 지속적인 통증이 있어요. 혈전이 있는지 검사를 받아봤는데 없었어요. 또한 등 아래쪽에 관절염이 있다는 결과가 나왔어요. 때로는 오른쪽으로 누울 수 없을 정도로 통증이 있어요. 그 통증은 허벅지에 있어요. 주로 자러 갈 때에만 나타나요.\n의사: 허리 통증과 허벅지로 전이되는 것은 싸이아티카로 해석될 수 있는데, 싸이아티카는 신경이 경로 상에서 압박을 받아 통증이 발생하는 것입니다. 보존적으로 치료하기 위해 딱딱한 침대에서 누우며 무릎 아래에 작은 쿠션을 놓는 것은 신경에 가해지는 압력을 줄여주어 통증을 완화시킬 것입니다. 여행 중에는 요추 보호대를 사용하여 추가적인 외상을 방지하고, 혈액 칼슘, 비타민 D 및 비타민 B12 수치를 점검해보세요. 이러한 수치가 정상보다 낮다면 상태를 악화시킬 수 있습니다. 일이 여행을 요구하는 경우 가벼운 짐을 가지세요. 식단에는 녹색 야채와 액체를 많이 섭취하고, 비타민 B12 보충제를 복용하세요(비타민 B12는 신경 토닉입니다). 가벼운 운동과 저출력 에어로빅(자전거 타기 및 수영)을 하고, 올바른 자세와 작업 환경을 유지하세요(잘못된 자세는 상태를 악화시킵니다). 통증이 주로 자러 갈 때 발생한다면 딱딱한 침대에서 자거나 반인치 메모리 폼 매트리스로</끝>\n환자: 자려고 할 때 통증이 주로 발생하는데, 어떻게 대처해야 할까요?\n의사: 통증을 완화시키기 위해 딱딱한 침대에서 누우며 무릎 아래에 작은 쿠션을 놓는 것이 좋습니다. 또한 혈액 검사를 통해 칼슘, 비타민 D 및 비타민 B12 수치를 확인하고 보충제를 복용하는 것이 도움이 될 수 있습니다. 가벼운 운동과 올바른 자세 유지도 중요합니다.</끝>\n환자: 그렇군요. 감사합니다.\n의사: 언제든지 도움이 필요하면 말씀해주세요. 건강하게 지내세요.</끝>\n환자: 감사합니다. 안녕히 계세요.\n의사: 안녕히 가세요.</끝>'

In [7]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [10]:
tokenNum_ai = 33687     # "의사"
tokenNum_human = 35604   # "환자"
tokenNum_com = 29901        # ":"

In [13]:
import transformers
from transformers import Trainer
import numpy as np

class maskTrainer(Trainer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def compute_loss(self, model, inputs, return_outputs=False):
    for x in range(len(inputs['labels'])):
      # print(tokenizer.decode(inputs['labels'][x]))

      maskindex1 = (inputs['labels'][x]==tokenNum_human).nonzero()[:, 0].cpu()
      temp = 0
      for i, index in enumerate(maskindex1):
        if (inputs['labels'][x][index+1] != tokenNum_com):
          maskindex1 = np.delete(maskindex1, i-temp)
          temp += 1

      maskindex2 = (inputs['labels'][x]==tokenNum_ai).nonzero()[:, 0].cpu()
      temp = 0
      for i, index in enumerate(maskindex2):
        if (inputs['labels'][x][index+1] != tokenNum_com):
          maskindex2 = np.delete(maskindex2, i-temp)
          temp += 1

      for i in range(len(maskindex1)):
        ai_index = -1
        for num in maskindex2:
          if (maskindex1[i] < num):
            ai_index = num
            break
        if (ai_index == -1):
          inputs['labels'][x][maskindex1[i]+2:] = -100
        else:
          inputs['labels'][x][maskindex1[i]+2:ai_index+2] = -100
    # print(inputs['labels'][x])

    outputs = model(**inputs)

    loss = outputs['loss']

    return (loss,outputs) if return_outputs else loss

In [ ]:
# import transformers

# # needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = maskTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        #max_steps=3000, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        fp16=True,
        output_dir="outputs",
        save_total_limit=2,
        logging_steps=50,
        report_to=["tensorboard"],
        num_train_epochs = 5,
        learning_rate=3e-4,
        # resume_from_checkpoint="./outputs/checkpoint-9500",
        # resume_from_checkpoint=True,
        lr_scheduler_type= "cosine",
        #optim="paged_adamw_8bit"
    
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()



/home/issac/anaconda3/envs/jupyter/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,1.659000
100,1.499900
150,1.409800
200,1.499500


In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.save_pretrained("./saved/doctor/7B/try2_5epoch")

In [22]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [ ]:
prompt = """아래는 전문적인 의사와 환자의 진료 기록이다.

환자: 갑자기 무릎이 아파요.
의사: 언제부터 그런 증상이 있었나요?</끝>
환자: 조금 전 부터에요. 전에 이 부위가 골절된 적이 있었는데. 지금은 전부 치료되었거든요. 왜 또 아플까요?
의사: 무릎을 움직일 때 통증이 심해지나요?</끝>
환자: 네. 무릎에 힘을 주면 깨질듯이 아파요.
의사: 무릎을 움직이는 것은 가능한가요?</끝>
환자: 음...못 움직입니다. 너무 아파요
의사:"""

tokenizer.decode(model.generate(
    **tokenizer(
        prompt, 
        return_tensors='pt', 
        return_token_type_ids=False
    ),
    max_new_tokens=500,
    temperature=0.2,
    no_repeat_ngram_size=10,
    early_stopping=True,
    eos_token_id=2,
    stopping_criteria=stopping_criteria
)[0]).replace(prompt+" ", "")